In [ ]:
pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=862a66ec5bc173acf4efd1432064cc7592f6744032c8fe7dca895dceb798aa0f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
# Code will record audio for 5 seconds and transcribe audio

from IPython.display import HTML, Audio
from google.colab import output
from base64 import b64decode
from google.colab import files

from google.colab import userdata
from openai import OpenAI

client = OpenAI(api_key = userdata.get('OPEN_API_KEY'))

# JavaScript to record audio from the browser and send the audio data back to Python
record_js_code = """
<script>
const recordAudio = () => {
  return new Promise(async resolve => {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const mediaRecorder = new MediaRecorder(stream);
    let audioChunks = [];

    mediaRecorder.addEventListener("dataavailable", event => {
      audioChunks.push(event.data);
    });

    const start = () => mediaRecorder.start();

    const stop = () => {
      return new Promise(resolve => {
        mediaRecorder.addEventListener("stop", () => {
          const audioBlob = new Blob(audioChunks);
          const audioUrl = URL.createObjectURL(audioBlob);
          const audio = new Audio(audioUrl);
          const play = () => audio.play();
          resolve({ audioBlob, audioUrl, play });
        });

        mediaRecorder.stop();
      });
    };

    resolve({ start, stop });
  });
};

const sleep = time => new Promise(resolve => setTimeout(resolve, time));

const record = async (time) => {
  const recorder = await recordAudio();
  recorder.start();
  await sleep(time);
  const audio = await recorder.stop();
  const reader = new FileReader();
  reader.readAsDataURL(audio.audioBlob);
  reader.onload = () => {
    const base64AudioMessage = reader.result.split(",")[1];
    google.colab.kernel.invokeFunction('notebook.SaveAudio', [base64AudioMessage], {});
  };
};

record(5000);
</script>
"""

# Python callback function to save the recorded audio
def save_audio(audio_base64):
  audio_bytes = b64decode(audio_base64)
  audio_file_path = 'recorded_audio.webm'  # Change the extension based on your needs
  with open(audio_file_path, 'wb') as f:
    f.write(audio_bytes)

  # Download audio to files
  files.download(audio_file_path)

  # Perform speech-to-text transcription after saving the file
  transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=open(audio_file_path, "rb"),  # Use the saved file for transcription
    response_format="text"
  )

  # Display results
  print(transcript)

# Register the Python callback function
output.register_callback('notebook.SaveAudio', save_audio)

# Display the JavaScript code to start recording
display(HTML(record_js_code))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Hi, my name is Josefina.



In [ ]:
# Code will record audio for 5 seconds and translate text to english

from IPython.display import HTML, Audio
from google.colab import output
from base64 import b64decode
from google.colab import files
from langdetect import detect

from google.colab import userdata
from openai import OpenAI

client = OpenAI(api_key = userdata.get('OPEN_API_KEY'))

# JavaScript to record audio from the browser and send the audio data back to Python
record_js_code = """
<script>
const recordAudio = () => {
  return new Promise(async resolve => {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const mediaRecorder = new MediaRecorder(stream);
    let audioChunks = [];

    mediaRecorder.addEventListener("dataavailable", event => {
      audioChunks.push(event.data);
    });

    const start = () => mediaRecorder.start();

    const stop = () => {
      return new Promise(resolve => {
        mediaRecorder.addEventListener("stop", () => {
          const audioBlob = new Blob(audioChunks);
          const audioUrl = URL.createObjectURL(audioBlob);
          const audio = new Audio(audioUrl);
          const play = () => audio.play();
          resolve({ audioBlob, audioUrl, play });
        });

        mediaRecorder.stop();
      });
    };

    resolve({ start, stop });
  });
};

const sleep = time => new Promise(resolve => setTimeout(resolve, time));

const record = async (time) => {
  const recorder = await recordAudio();
  recorder.start();
  await sleep(time);
  const audio = await recorder.stop();
  const reader = new FileReader();
  reader.readAsDataURL(audio.audioBlob);
  reader.onload = () => {
    const base64AudioMessage = reader.result.split(",")[1];
    google.colab.kernel.invokeFunction('notebook.SaveAudio', [base64AudioMessage], {});
  };
};

record(5000);
</script>
"""

# Python callback function to save the recorded audio
def save_audio(audio_base64):
  audio_bytes = b64decode(audio_base64)
  audio_file_path = 'recorded_audio.webm'  # Change the extension based on your needs
  with open(audio_file_path, 'wb') as f:
    f.write(audio_bytes)

  # Download audio to files
  files.download(audio_file_path)

  # Translate to english
  translation = client.audio.translations.create(
    model="whisper-1",
    file=open(audio_file_path, "rb"),
    response_format="text"
  )

  # Display results
  print(translation)

# Register the Python callback function
output.register_callback('notebook.SaveAudio', save_audio)

# Display the JavaScript code to start recording
display(HTML(record_js_code))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Hi, my name is Josefina.

